In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv


In [2]:
test_path='../input/house-prices-advanced-regression-techniques/test.csv'
train_path='../input/house-prices-advanced-regression-techniques/train.csv'
test = pd.read_csv(test_path)
train = pd.read_csv(train_path)

In [3]:
train.shape
#test.shape
#train=train.drop(columns="LotFrontage")

(1460, 81)

In [4]:
test.shape

(1459, 80)

In [5]:
print(train.isnull().sum())

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64


In [6]:
#handling the missing data with NA values(more than 80%)
train=train.drop(["Alley","PoolQC","Fence","MiscFeature"],axis=1)



In [7]:
train.shape

(1460, 77)

In [8]:
#handling the missing data with NA values(less than 80%)

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
#imputer = imputer.fit(train[:,])
imputer=imputer.fit(train[['LotFrontage','FireplaceQu','GarageYrBlt','GarageQual','GarageCond','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical','GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond','MasVnrArea']])
train[['LotFrontage','FireplaceQu','GarageYrBlt','GarageQual','GarageCond','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical','GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond','MasVnrArea']]=imputer.transform(train[['LotFrontage','FireplaceQu','GarageYrBlt','GarageQual','GarageCond','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical','GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond','MasVnrArea']])





In [9]:
#checking once more for na values
missing_val_count_by_column = (train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


In [10]:
#filtering the categorical columns from the data set
categorical_feature_mask = train.dtypes==object

In [11]:
#transfering categorical columns into a list
categorical_cols = train.columns[categorical_feature_mask].tolist()
train[categorical_cols].head(10)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
5,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,...,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,WD,Normal
6,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,...,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,WD,Normal
7,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,...,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
8,RM,Pave,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,...,TA,Min1,TA,Detchd,Unf,Fa,TA,Y,WD,Abnorml
9,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,...,TA,Typ,TA,Attchd,RFn,Gd,TA,Y,WD,Normal


In [12]:
#Encoding the categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder_train=LabelEncoder()
train[categorical_cols] = train[categorical_cols].apply(lambda col: labelencoder_train.fit_transform(col))

In [13]:
X=train.iloc[:,0:76]
y=train.iloc[:,76]

In [14]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=1000,random_state = 1)
regressor.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [15]:
# Fitting SVR to the dataset
#from sklearn.svm import SVR
#regressor = SVR(kernel = 'rbf')
#regressor.fit(X, y)

In [16]:
 #Fitting Decision Tree Regression to the dataset
#from sklearn.tree import DecisionTreeRegressor
#regressor = DecisionTreeRegressor(random_state = 0)
#regressor.fit(X, y)

In [17]:
 #Fitting Simple Linear Regression to the Training set
#from sklearn.linear_model import LinearRegression
#regressor = LinearRegression()
#regressor.fit(X, y)

In [18]:
#ElasticNet Regression
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
X, y = make_regression(n_features=76, random_state=0)
regressor = ElasticNet(random_state=0)
regressor.fit(X, y)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=0, selection='cyclic', tol=0.0001, warm_start=False)

In [19]:
#Data processing for test data

missing_val_count_by_column = (test.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64


In [20]:
#handling the missing data with NA values(more than 80%)
test=test.drop(["Alley","PoolQC","Fence","MiscFeature"],axis=1)

In [21]:
test.shape

(1459, 76)

In [22]:
#handling the missing data with NA values(less than 80%)

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
#imputer = imputer.fit(train[:,])
imputer=imputer.fit(test[['GarageArea','Exterior2nd','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath','KitchenQual','KitchenQual','Functional','GarageCars','SaleType','MSZoning','LotFrontage','FireplaceQu','GarageYrBlt','GarageQual','GarageCond','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical','GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond','MasVnrArea','Utilities','Exterior1st']])
test[['GarageArea','Exterior2nd','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath','KitchenQual','KitchenQual','Functional','GarageCars','SaleType','MSZoning','LotFrontage','FireplaceQu','GarageYrBlt','GarageQual','GarageCond','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical','GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond','MasVnrArea','Utilities','Exterior1st']]=imputer.transform(test[['GarageArea','Exterior2nd','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath','KitchenQual','KitchenQual','Functional','GarageCars','SaleType','MSZoning','LotFrontage','FireplaceQu','GarageYrBlt','GarageQual','GarageCond','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical','GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond','MasVnrArea','Utilities','Exterior1st']])





In [23]:
categorical_feature_mask = test.dtypes==object

In [24]:
#transfering categorical columns into a list
categorical_cols = test.columns[categorical_feature_mask].tolist()
test[categorical_cols].head(10)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,...,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,WD,Normal
1,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,Gd,Typ,Gd,Attchd,Unf,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,TA,Typ,TA,Attchd,Fin,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,Gd,Typ,Gd,Attchd,Fin,TA,TA,Y,WD,Normal
4,RL,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,...,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,WD,Normal
5,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,...,TA,Typ,TA,Attchd,Fin,TA,TA,Y,WD,Normal
6,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,TA,Typ,Gd,Attchd,Fin,TA,TA,Y,WD,Normal
7,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,TA,Typ,Gd,Attchd,Fin,TA,TA,Y,WD,Normal
8,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,Gd,Typ,Po,Attchd,Unf,TA,TA,Y,WD,Normal
9,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,TA,Typ,Gd,Attchd,Fin,TA,TA,Y,WD,Normal


In [25]:
#Encoding the categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder_test=LabelEncoder()
test[categorical_cols] = test[categorical_cols].apply(lambda col: labelencoder_test.fit_transform(col))

In [26]:
#predicting new result

train_predictions=regressor.predict(X)


In [27]:
train_predictions

array([ -94.64881492,   23.22342854,  -67.8376268 ,  180.38627098,
       -339.92101269,  -76.86268665,   81.98125908,  -39.80896287,
       -136.92754783, -220.72096173,  -83.02114864,  -90.96001498,
        -55.06537713,   27.98079993,  143.45949729,  218.32489263,
       -165.78638122,   84.35213591, -177.54730381,  223.00292567,
         33.39496714, -184.56172768,  247.26914161,  129.29373947,
       -154.59305852,  -21.33853769, -149.28413166,  -15.52895749,
         64.04296655,   80.61068428, -107.08122787,  110.02864931,
        -86.7127883 ,   34.63190763,  115.65727938, -214.64876424,
        138.79688933,  -71.50890197,  -78.42835555,  -40.23513059,
        118.57260658,  -48.91428512,  242.98413522, -134.93908055,
         48.13839841,  -97.6818908 ,  -54.99538792,  126.37183469,
        313.0030097 , -162.6845769 ,  129.52368322,  180.29417046,
         58.14285306,   30.10592995,  -57.00844758,   70.35429782,
        -81.57919802,  -97.28765193,  208.55580163,  -42.71149

In [28]:
from sklearn.metrics import mean_absolute_error

In [29]:
mae=mean_absolute_error(train_predictions,y)
mae

49.71900534600016

In [30]:
test_predictions=regressor.predict(test)

In [31]:
test_predictions

array([ -9777.27541853,  65849.49484332, -11164.18841846, ...,
        -1542.1527244 ,  12543.99859433,  20811.04392062])

In [32]:
output = pd.DataFrame({'Id': test.Id,
                       'SalePrice': test_predictions})
output.to_csv('submission_RF_v_3.csv', index=False)

In [33]:
#submission['Id']=test['Id'].to_numpy()

In [34]:
#submission.head(20)

In [35]:
#submission['SalePrice']=test_predictions

In [36]:
#submission.to_csv(r'Users\baljindersmagh\Desktop\FinalSubmission.csv')

In [37]:
#print(submission)